# Worksheet 07

Name: Yingjie Ouyang  
UID: U86074230 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [ ]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[3.3055984886409684, 4.655116228257954, 4.452668282508901, 6.6811478024877875, 4.173826866765857, 5.790206314269063, 4.848784943960292, 5.667773160161924, 4.691199199587158, 6.625822230922454]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [ ]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[7.030069693246384, 8.732465012625477, 8.627035775294003, 7.218006926069083, 7.814285366763424, 7.760242148085559, 7.837158928872747, 7.737096197823686, 7.57954830484593, 5.900519992309928]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [ ]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.625822230922454, 5.900519992309928, 7.57954830484593, 4.691199199587158, 7.737096197823686, 7.837158928872747, 5.667773160161924, 4.848784943960292, 5.790206314269063, 4.173826866765857]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Mean1 = 5  
Mean2 = 8  
Stdev1 = 1  
Stdev2 = 2   
P(C_1) = 0.5  
P(C_2) = 0.5

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [ ]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.900519992309928, 4.691199199587158, 5.667773160161924, 4.848784943960292, 5.790206314269063, 4.173826866765857]
[6.625822230922454, 7.57954830484593, 7.737096197823686, 7.837158928872747]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 5.178718412842371,  mean_2 = 7.444906415616204
var_1 = 0.4150712693577434,  var_2 = 0.2320660607333263


Pretty close to the true values, may be dependent on the samples we chose.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [ ]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  6.625822230922454
probability of observing that point if it came from cluster 0 =  0.002204795610606134
probability of observing that point if it came from cluster 1 =  0.0033897562138826614
point =  5.900519992309928
probability of observing that point if it came from cluster 0 =  0.21189526358090194
probability of observing that point if it came from cluster 1 =  4.151675339415515e-10
point =  7.57954830484593
probability of observing that point if it came from cluster 0 =  5.222183511917164e-08
probability of observing that point if it came from cluster 1 =  1.4527898589233195
point =  4.691199199587158
probability of observing that point if it came from cluster 0 =  0.48219360599687144
probability of observing that point if it came from cluster 1 =  4.573938945187746e-31
point =  7.737096197823686
probability of observing that point if it came from cluster 0 =  5.408592644104536e-09
probability of observing that point if it came from cluster 1 =  0.778141569699048
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [ ]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6493835858064638,  P(C_2) = 0.35061641419353623
mean_1 = 5.288766132026775,  mean_2 = 7.560272683586877
var_1 = 0.42718176985543305,  var_2 = 0.24537543651880744


I ran the code above multiple times, and it seems to either perform better or worse depending on the values we randomly chose. Overall, they stay around the same as kmeans.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [ ]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  6.625822230922454
probability of observing that point if it came from cluster 0 =  0.006966268854429181
probability of observing that point if it came from cluster 1 =  0.001153046017568211
point =  5.900519992309928
probability of observing that point if it came from cluster 0 =  0.3349399622024166
probability of observing that point if it came from cluster 1 =  1.8872372375746418e-10
point =  7.57954830484593
probability of observing that point if it came from cluster 0 =  5.31880921955346e-07
probability of observing that point if it came from cluster 1 =  1.620835603563353
point =  4.691199199587158
probability of observing that point if it came from cluster 0 =  0.35106085674707427
probability of observing that point if it came from cluster 1 =  3.337733959057534e-30
point =  7.737096197823686
probability of observing that point if it came from cluster 0 =  6.876273923908426e-08
probability of observing that point if it came from cluster 1 =  1.2540490896648842
point =  7

The probabilities have stayed around the same, there are no extreme changes.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [ ]:
labels = []
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
  if p[1] > p[2]:
    labels.append(0)
  else:
    labels.append(1)

print(labels)

label_pairs = zip(data, labels)
for p in label_pairs:
  print("point", p[0], "was assigned to cluster", p[1])

[0, 0, 1, 0, 1, 1, 0, 0, 0, 0]
point 6.625822230922454 was assigned to cluster 0
point 5.900519992309928 was assigned to cluster 0
point 7.57954830484593 was assigned to cluster 1
point 4.691199199587158 was assigned to cluster 0
point 7.737096197823686 was assigned to cluster 1
point 7.837158928872747 was assigned to cluster 1
point 5.667773160161924 was assigned to cluster 0
point 4.848784943960292 was assigned to cluster 0
point 5.790206314269063 was assigned to cluster 0
point 4.173826866765857 was assigned to cluster 0


Remaining to be completed tomorrow!

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.